In [11]:
from model import GeoCLIP
from model import ImageEncoder
from model import LocationEncoder
from train import train
from train import dataloader
import os
from torch import nn
import torch
import torch.nn.functional as F


In [ ]:
class SimCLRLoss(nn.Module):

    def __init__(self, batch_size, temperature):
        self.batch_size = batch_size
        self.temperature = temperature
        self.mask = (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float()


    def calc_similarity_batch(self, a, b):
       representations = torch.cat([a, b], dim=0)
       return F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)

        
    def forward(self, proj_1, proj_2):
        """
        proj_1 and proj_2 are batched embeddings [batch, embedding_dim]
        where corresponding indices are pairs
        z_i, z_j in the SimCLR paper
        """
        batch_size = proj_1.shape[0]
        z_i = F.normalize(proj_1, p=2, dim=1)
        z_j = F.normalize(proj_2, p=2, dim=1)

        similarity_matrix = self.calc_similarity_batch(z_i, z_j)

        # Postive similarities are on the off-diagonals
        sim_ij = torch.diag(similarity_matrix, batch_size)
        sim_ji = torch.diag(similarity_matrix, -batch_size)

        positives = torch.cat([sim_ij, sim_ji], dim=0)

        nominator = torch.exp(positives / self.temperature)

        # Mask out the main diagonal and calculate the softmax
        denominator = (self.mask, similarity_matrix) * torch.exp(similarity_matrix / self.temperature)

        all_losses = -torch.log(nominator / torch.sum(denominator, dim=1))
        loss = torch.sum(all_losses) / (2 * self.batch_size)
        return loss

In [9]:
#loader = dataloader.GeoDataLoader("data/geojsons", "data/images")
geo_clip = GeoCLIP()
#train(train_dataloader=loader, model=geo_clip)

#geo_clip.save("model")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
model.safetensors: 100%|██████████| 1.71G/1.71G [02:39<00:00, 10.7MB/s]
tokenizer_config.json: 100%|██████████| 905/905 [00:00<00:00, 4.24MB/s]
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
vocab.json: 100%|█

In [10]:
geo_clip.predict("/workspace/geoclip/images/Kauai.png", 10)

(tensor([[  22.1980, -159.6219],
         [  22.1785, -159.6501],
         [  22.1759, -159.6542],
         [  22.1751, -159.6559],
         [  22.1502, -159.6636],
         [  22.2178, -159.5888],
         [  21.4986, -158.1512],
         [  22.2207, -159.5827],
         [  21.3382, -157.8054],
         [  20.1172, -155.5842]]),
 tensor([0.0731, 0.0704, 0.0679, 0.0669, 0.0527, 0.0454, 0.0370, 0.0314, 0.0293,
         0.0288]))

In [16]:
optim = torch.optim.SGD(geo_clip.parameters(), lr=0.1)
criterion = SimCLRLoss(10, 0.1)
EPOCHS = 2
BATCH_SIZE = 2
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.1)

In [13]:
train(train_dataloader, geo_clip, criterion, optim, scheduler, epoch = EPOCHS, batch_size =BATCH_SIZE, device = "cpu")

NameError: name 'train_dataloader' is not defined